<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [64]:
import numpy as np

In [65]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [66]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [67]:
def extract_vocabulary(corpus):
    tokenized_documents = [doc.lower().split() for doc in corpus]
    all_tokens = [token for doc_tokens in tokenized_documents for token in doc_tokens]
    vocabulary = np.unique(all_tokens)
    vocab_list = vocabulary.tolist()
    return vocab_list

vocab_list = extract_vocabulary(corpus)
vocab_list

['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que']

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [68]:

def one_hot_encoding(vocab, docs):

    tokenized_documents = [doc.lower().split() for doc in corpus]

    word_to_index = {word: idx for idx, word in enumerate(vocab)}

    num_words = len(vocab)
    num_documents = len(docs)
    one_hot_matrix = np.zeros((num_documents, num_words))

    for doc_idx, doc_tokens in enumerate(tokenized_documents):
        for token in doc_tokens:
            if token in word_to_index:
                word_idx = word_to_index[token]
                one_hot_matrix[doc_idx, word_idx] = 1
    
    return one_hot_matrix

one_hot_encoding(vocab_list, corpus)

array([[0., 1., 0., 1., 0., 1., 0., 0., 1.],
       [1., 1., 1., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 1., 0.]])

### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [69]:
def word_freq(vocab, docs):

    num_documents = len(docs)
    word_to_index = {word: idx for idx, word in enumerate(vocab)}

    word_freq_dict = [{word: 0 for word in vocab} for _ in range(num_documents)]

    for doc_idx, doc_tokens in enumerate(tokenized_documents):
        for token in doc_tokens:
            if token in word_to_index:
                word_idx = word_to_index[token]
                word_freq_dict[doc_idx][token] += 1

    frequency_vectors = np.array([[word_freq_dict[doc_idx][word] for word in vocab] for doc_idx in range(num_documents)])

    return frequency_vectors

word_freq(vocab_list, corpus)

array([[0, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 1, 1, 1, 0, 1, 2, 0, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0]])

### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [70]:
def calculate_tfidf(vocab, docs):
    
    num_documents = len(docs)
    num_words = len(vocab)
    
    tf_matrix = word_freq(vocab, docs)
    idf_vector = np.array([np.log(num_documents / np.count_nonzero(tf_matrix[:, word_idx])) for word_idx in range(num_words)])
    tfidf_matrix = tf_matrix * idf_vector
    
    return tfidf_matrix

tfidf_matrix = calculate_tfidf(vocab_list, corpus)
tfidf_matrix

array([[0.        , 0.40546511, 0.        , 0.40546511, 0.        ,
        0.40546511, 0.        , 0.        , 1.09861229],
       [1.09861229, 0.40546511, 1.09861229, 0.40546511, 0.        ,
        0.40546511, 0.81093022, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.09861229,
        0.        , 0.40546511, 1.09861229, 0.        ]])

### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [75]:
def sort_documents_by_cosine_similarity(corpus, tfidf_matrix, doc_index):
    doc_tfidf = tfidf_matrix[doc_index]
    cosine_similarities = [cosine_similarity(doc_tfidf, tfidf_matrix[i]) for i in range(len(corpus))]
    sorted_indices = np.argsort(cosine_similarities)[::-1]
    sorted_indices = sorted_indices[sorted_indices != doc_index]
    sorted_documents = [corpus[idx] for idx in sorted_indices]
    return sorted_documents


document_index = 0
similar_documents = sort_documents_by_cosine_similarity(corpus, tfidf_matrix, document_index)

print(f"Original Document: {corpus[document_index]}")

for idx, doc in enumerate(similar_documents, start=1):
    print(f"Similar Document {idx}: {doc}")

Original Document: que dia es hoy
Similar Document 1: martes el dia de hoy es martes
Similar Document 2: martes muchas gracias
